In [1]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')

## Big M method

In [2]:

M=10**3#big M

objective="min"
m=2#no of equations
n=3#no of variables
a=[[1,4,2,'>='],[3,1,2,">="]]
b=[5,4]
Coeff=[2,9,1]

In [3]:
def convert_to_maximization_type(a,b,Coeff,m,n,objective):
    if objective != 'max':
        Coeff=[-Coeff[i] for i in range(n)]
    for i in range(m):
        if b[i] < 0:
            for j in range(n):
                a[i][j]=-a[i][j]
            b[i]=-b[i]
    return a,b,Coeff,m,n
                
                
    

In [4]:


#1
def initialize_bigM_table(a, b, m, n, Coeff,objective):
    data={}
    col=['x'+str(i+1) for i in range(n)]
    convert_dict={'x'+str(i+1):float for i in range(n)}
    jconvert_dict={'x'+str(i+1):float for i in range(n)}
    a,b,Coeff,m,n=convert_to_maximization_type(a,b,Coeff,m,n,objective)
    Cj_dict={'x'+str(i+1):[Coeff[i]] for i in range(n)}
    Zj_dict={'x'+str(i+1):[0] for i in range(n)}
    
    for i in range(m):
        if a[i][-1] == "<=":
            Cj_dict['sl'+str(i+1)]=[0]
            Zj_dict['sl'+str(i+1)]=[0]
            convert_dict['sl'+str(i+1)]=float
            jconvert_dict['sl'+str(i+1)]=float
            col.append('sl'+str(i+1))
        elif a[i][-1] == "=":
            Cj_dict['a'+str(i+1)]=[-M]
            Zj_dict['a'+str(i+1)]=[0]
            convert_dict['a'+str(i+1)]=float
            jconvert_dict['a'+str(i+1)]=float
            col.append('a'+str(i+1))
        else:
            Cj_dict['sr'+str(i+1)]=[0]
            Zj_dict['sr'+str(i+1)]=[0]
            convert_dict['sr'+str(i+1)]=float
            jconvert_dict['sr'+str(i+1)]=float
            col.append('sr'+str(i+1))
            Cj_dict['a'+str(i+1)]=[-M]
            Zj_dict['a'+str(i+1)]=[0]
            convert_dict['a'+str(i+1)]=float
            jconvert_dict['a'+str(i+1)]=float
            col.append('a'+str(i+1))
            
    Cj_dict['sol']=0
    Zj_dict['sol']=0   
    convert_dict['sol']=float
    jconvert_dict['sol']=float
    #Cj_dict.extend({0 for i in range(m)])
    #Zj=[0 for i in range(m+n)]
    data['CBi']=[]    
    convert_dict['CBi']=float    
    data['basic_var']=[]    
    convert_dict['basic_var']=str
    for i in range(n):
        data['x'+str(i+1)]=[a[j][i] for j in range(m)]
    #augmenting data with slack variables, surplus variables and artificial variables
    for i in range(m):
        if a[i][-1] == "<=":
            data['sl'+str(i+1)]=[float(i==j) for j in range(m)]
            data['basic_var'].append('sl'+str(i+1))
            data['CBi'].append(0)
        elif a[i][-1] == "=":
            data['a'+str(i+1)]=[float(i==j) for j in range(m)]
            data['basic_var'].append('a'+str(i+1))
            data['CBi'].append(-M)
        else:
            data['sr'+str(i+1)]=[-1*float(i==j) for j in range(m)]
            data['a'+str(i+1)]=[float(i==j) for j in range(m)]
            data['basic_var'].append('a'+str(i+1))
            data['CBi'].append(-M)
        
    data['sol']=[b[j] for j in range(m)]
    data['ratio']=[0 for j in range(m)]
    convert_dict['ratio']=float
    df = pd.DataFrame(data, index=[str(i+1) for i in range(m)]) 
    Zj = pd.DataFrame(Zj_dict, index=['Zj']) 
    row=list(df.index)
    col.append('sol')
    for j in col:
        Zj[j]['Zj']=sum([df[j][i]*df['CBi'][i] for i in row]) 
    Cj = pd.DataFrame(Cj_dict, index=['Cj'])    
    df = df.astype(convert_dict)     
    Zj = Zj.astype(jconvert_dict)
    Cj = Cj.astype(jconvert_dict)
    return df,Cj,Zj,col
  

In [5]:
#2 
def print_table(df, Cj, Zj):
    print("simplex table:")
    frames = [df, Cj, Zj]
    result = pd.concat(frames)
    
    print(result)
#     print("Cj's: ",Cj)
    
#     print("Zj's: ",Zj)
    
    
  
 
df,Cj,Zj,col = initialize_bigM_table(a, b, m, n, Coeff,objective)
print_table(df,Cj,Zj)

simplex table:
       CBi basic_var      x1      x2      x3     sr1      a1     sr2      a2  \
1  -1000.0        a1     1.0     4.0     2.0    -1.0     1.0    -0.0     0.0   
2  -1000.0        a2     3.0     1.0     2.0    -0.0     0.0    -1.0     1.0   
Cj     NaN       NaN    -2.0    -9.0    -1.0     0.0 -1000.0     0.0 -1000.0   
Zj     NaN       NaN -4000.0 -5000.0 -4000.0  1000.0 -1000.0  1000.0 -1000.0   

       sol  ratio  
1      5.0    0.0  
2      4.0    0.0  
Cj     0.0    NaN  
Zj -9000.0    NaN  


In [6]:
#3
def Zj_Cj(Cj, Zj,col):    
    return  {i:Zj[i]['Zj']-Cj[i]['Cj'] for i in col}


In [7]:
#4
# def col_name(i,n):
#     if (i+1)<=n:
#         return 'x'+str(i+1)
#     else:
#         return 's'+str(i+1-n)
    
def get_key_column(zj_cj, m, n,col):
    val_min=zj_cj[col[0]]
    idx_min=col[0] 
    for i in col[:-1]:
        if val_min>zj_cj[i]:
            val_min=zj_cj[i]
            idx_min=i
    
    return idx_min
    


def get_key_row(df, key_col, m):   
    df['ratio']=df['sol']/df[key_col]
    row=list(df.index)
    val_min=999999999
    idx_min='1'
    for i in row:
        if val_min>df['ratio'][i]>=0:
            val_min=df['ratio'][i]
            idx_min=i
    return df, idx_min, val_min




In [8]:
print("_____output_________")
zj_cj=Zj_Cj(Cj, Zj, col)
key_col=get_key_column(zj_cj, m, n,col)
df, key_row, min_ratio = get_key_row(df, key_col, m)
print(df)
print()
print("Key column is " )
print(df[key_col])
print()
print("Minimum ratio is ", min_ratio)
print()
print("Key row is ")
print(df.loc[key_row])
print()
print("Key element is ", df[key_col][key_row])


_____output_________
      CBi basic_var   x1   x2   x3  sr1   a1  sr2   a2  sol  ratio
1 -1000.0        a1  1.0  4.0  2.0 -1.0  1.0 -0.0  0.0  5.0   1.25
2 -1000.0        a2  3.0  1.0  2.0 -0.0  0.0 -1.0  1.0  4.0   4.00

Key column is 
1    4.0
2    1.0
Name: x2, dtype: float64

Minimum ratio is  1.25

Key row is 
CBi         -1000
basic_var      a1
x1              1
x2              4
x3              2
sr1            -1
a1              1
sr2            -0
a2              0
sol             5
ratio        1.25
Name: 1, dtype: object

Key element is  4.0


In [9]:
def modify_table(key_row, key_col, Cj, Zj,df,col):
    df_new = df.copy(deep=True)
    Zj_new = Zj.copy(deep=True)
    col_len=len(df[key_col])
    row_len=len(df.loc[key_row])
    p=key_elem=df[key_col][key_row]
    entering_var=key_col
    leaving_var=df['basic_var'][key_row]
    print("Entering variable is ", entering_var)
    print("Leaving variable is ", leaving_var)
    df_new['basic_var'][key_row]=entering_var
    #swap the coefficients value
    df_new['CBi'][key_row]=Cj[key_col]['Cj']    
    row=list(df.index)
    
    for j in col:
        r=df[j][key_row]              
        for i in row:    
            if j==key_col:
                df_new[j][i]=(-df[j][i])/p                
            elif i==key_row:
                df_new[j][i]=df[j][i]/p               
            else:
                s=df[j][i]
                q=df[key_col][i] 
                df_new[j][i]=(p*s-q*r)/p
    #print(1/p)
    df_new[key_col][key_row]=1/p
    #df_new[key_col]=df[]
    df_new[leaving_var]  = df_new[entering_var]    
    df_new[entering_var] = df[leaving_var]
    #df_new[entering_var][key_row]=1
    #modify zj
    for j in col:
        Zj_new[j]['Zj']=sum([df_new[j][i]*df_new['CBi'][i] for i in row]) 
        #Zj_new[j]['Zj']=16.0*0.05
    return df_new,Cj,Zj_new
    
    
    
    
    
    
    
    
    

In [10]:
df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df,col)
print_table(df_new,Cj,Zj_new)


Entering variable is  x2
Leaving variable is  a1
simplex table:
       CBi basic_var       x1   x2      x3     sr1       a1     sr2      a2  \
1     -9.0        x2     0.25  1.0     0.5   -0.25     0.25    -0.0     0.0   
2  -1000.0        a2     2.75  0.0     1.5    0.25    -0.25    -1.0     1.0   
Cj     NaN       NaN    -2.00 -9.0    -1.0    0.00 -1000.00     0.0 -1000.0   
Zj     NaN       NaN -2752.25 -9.0 -1504.5 -247.75   247.75  1000.0 -1000.0   

        sol  ratio  
1      1.25   1.25  
2      2.75   4.00  
Cj     0.00    NaN  
Zj -2761.25    NaN  


In [11]:
#Check is optimal solution is reached or not
def is_optimal(zj_cj):
    zj_cj_soless=zj_cj.copy()
    del zj_cj_soless['sol']
    val=min(zj_cj_soless.values())
    if val>=0:
        print("Optimality is reached")
        return True
    else:
        print("Lets proceed to next iteration")
        return False

In [12]:
def simplex_method(a, b, m, n, Coeff,objective):    
    df,Cj,Zj,col = initialize_bigM_table(a, b, m, n, Coeff,objective)
    zj_cj=Zj_Cj(Cj, Zj,col)
    key_col=get_key_column(zj_cj, m, n,col)
    df, key_row, min_ratio = get_key_row(df, key_col, m)
    print_table(df,Cj,Zj)
    if objective=="max":
        print("value of objective function is ",Zj['sol']['Zj'])
    else:
        print("value of objective function is ",-Zj['sol']['Zj'])
    i=0
    while not is_optimal(zj_cj) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return
        df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df,col)
        
        df=df_new
        Zj=Zj_new
        print_table(df,Cj,Zj)        
        zj_cj=Zj_Cj(Cj, Zj,col)
        key_col=get_key_column(zj_cj, m, n,col)
        df, key_row, min_ratio = get_key_row(df, key_col, m) 
        if objective=="max":
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            print("value of objective function is ",-Zj['sol']['Zj'])
            
        i+=1
        
    
    

In [13]:
#1
objective="min"
m=2#no of equations
n=3#no of variables
a=[[1,4,2,'>='],[3,1,2,">="]]
b=[5,4]
Coeff=[2,9,1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
       CBi basic_var      x1      x2      x3     sr1      a1     sr2      a2  \
1  -1000.0        a1     1.0     4.0     2.0    -1.0     1.0    -0.0     0.0   
2  -1000.0        a2     3.0     1.0     2.0    -0.0     0.0    -1.0     1.0   
Cj     NaN       NaN    -2.0    -9.0    -1.0     0.0 -1000.0     0.0 -1000.0   
Zj     NaN       NaN -4000.0 -5000.0 -4000.0  1000.0 -1000.0  1000.0 -1000.0   

       sol  ratio  
1      5.0   1.25  
2      4.0   4.00  
Cj     0.0    NaN  
Zj -9000.0    NaN  
value of objective function is  9000.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  a1
simplex table:
       CBi basic_var       x1   x2      x3     sr1       a1     sr2      a2  \
1     -9.0        x2     0.25  1.0     0.5   -0.25     0.25    -0.0     0.0   
2  -1000.0        a2     2.75  0.0     1.5    0.25    -0.25    -1.0     1.0   
Cj     NaN       NaN    -2.00 -9.0    -1.0    0.00 -1000.

In [14]:
#2
objective="min"
m=3#no of equations
n=2#no of variables
a=[[3,1,'>='],[1,1,">="],[1,2,">="]]
b=[27,21,30]
Coeff=[4,2]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
       CBi basic_var      x1      x2     sr1      a1     sr2      a2     sr3  \
1  -1000.0        a1     3.0     1.0    -1.0     1.0    -0.0     0.0    -0.0   
2  -1000.0        a2     1.0     1.0    -0.0     0.0    -1.0     1.0    -0.0   
3  -1000.0        a3     1.0     2.0    -0.0     0.0    -0.0     0.0    -1.0   
Cj     NaN       NaN    -4.0    -2.0     0.0 -1000.0     0.0 -1000.0     0.0   
Zj     NaN       NaN -5000.0 -4000.0  1000.0 -1000.0  1000.0 -1000.0  1000.0   

        a3      sol  ratio  
1      0.0     27.0    9.0  
2      0.0     21.0   21.0  
3      1.0     30.0   30.0  
Cj -1000.0      0.0    NaN  
Zj -1000.0 -78000.0    NaN  
value of objective function is  78000.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  a1
simplex table:
       CBi basic_var   x1           x2         sr1           a1     sr2  \
1     -4.0        x1  1.0     0.333333   -0.333333     0.333333 

In [15]:
#3
objective="max"
m=3#no of equations
n=2#no of variables
a=[[3,1,'='],[4,3,">="],[1,2,"<="]]
b=[3,6,4]
Coeff=[-2,-1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
       CBi basic_var      x1      x2      a1     sr2      a2  sl3     sol  \
1  -1000.0        a1     3.0     1.0     1.0    -0.0     0.0  0.0     3.0   
2  -1000.0        a2     4.0     3.0     0.0    -1.0     1.0  0.0     6.0   
3      0.0       sl3     1.0     2.0     0.0    -0.0     0.0  1.0     4.0   
Cj     NaN       NaN    -2.0    -1.0 -1000.0     0.0 -1000.0  0.0     0.0   
Zj     NaN       NaN -7000.0 -4000.0 -1000.0  1000.0 -1000.0  0.0 -9000.0   

    ratio  
1     1.0  
2     1.5  
3     4.0  
Cj    NaN  
Zj    NaN  
value of objective function is  -9000.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  a1
simplex table:
       CBi basic_var   x1           x2           a1     sr2      a2  sl3  \
1     -2.0        x1  1.0     0.333333     0.333333    -0.0     0.0  0.0   
2  -1000.0        a2  0.0     1.666667    -1.333333    -1.0     1.0  0.0   
3      0.0       sl3  0.0     1

In [16]:
#4
objective="max"
m=3#no of equations
n=2#no of variables
a=[[2,1,'>='],[1,3,"<="],[0,1,"<="]]
b=[2,3,4]
Coeff=[3,-1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
       CBi basic_var      x1      x2     sr1      a1  sl2  sl3     sol  ratio
1  -1000.0        a1     2.0     1.0    -1.0     1.0  0.0  0.0     2.0    1.0
2      0.0       sl2     1.0     3.0    -0.0     0.0  1.0  0.0     3.0    3.0
3      0.0       sl3     0.0     1.0    -0.0     0.0  0.0  1.0     4.0    inf
Cj     NaN       NaN     3.0    -1.0     0.0 -1000.0  0.0  0.0     0.0    NaN
Zj     NaN       NaN -2000.0 -1000.0  1000.0 -1000.0  0.0  0.0 -2000.0    NaN
value of objective function is  -2000.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  a1
simplex table:
    CBi basic_var   x1   x2  sr1      a1  sl2  sl3  sol  ratio
1   3.0        x1  1.0  0.5 -0.5     0.5  0.0  0.0  1.0    1.0
2   0.0       sl2  0.0  2.5  0.5    -0.5  1.0  0.0  2.0    3.0
3   0.0       sl3  0.0  1.0  0.0    -0.0  0.0  1.0  4.0    inf
Cj  NaN       NaN  3.0 -1.0  0.0 -1000.0  0.0  0.0  0.0    NaN
Zj  NaN    

In [17]:
#5
objective="max"
m=3#no of equations
n=4#no of variables
a=[[1,2,3,0,'='],[2,1,5,0,"="],[1,2,1,1,"="]]
b=[15,20,10]
Coeff=[1,2,3,-1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
       CBi basic_var      x1      x2      x3      x4      a1      a2      a3  \
1  -1000.0        a1     1.0     2.0     3.0     0.0     1.0     0.0     0.0   
2  -1000.0        a2     2.0     1.0     5.0     0.0     0.0     1.0     0.0   
3  -1000.0        a3     1.0     2.0     1.0     1.0     0.0     0.0     1.0   
Cj     NaN       NaN     1.0     2.0     3.0    -1.0 -1000.0 -1000.0 -1000.0   
Zj     NaN       NaN -4000.0 -5000.0 -9000.0 -1000.0 -1000.0 -1000.0 -1000.0   

        sol  ratio  
1      15.0    5.0  
2      20.0    4.0  
3      10.0   10.0  
Cj      0.0    NaN  
Zj -45000.0    NaN  
value of objective function is  -45000.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x3
Leaving variable is  a2
simplex table:
       CBi basic_var     x1      x2   x3      x4      a1      a2      a3  \
1  -1000.0        a1   -0.2     1.4  0.0     0.0     1.0    -0.6     0.0   
2      3.0        x3    0.4     0.

In [18]:
#6
objective="max"
m=2#no of equations
n=3#no of variables
a=[[1,-2,3,'='],[3,2,4,"="]]
b=[2,1]
Coeff=[-2,1,3]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
       CBi basic_var      x1   x2      x3      a1      a2     sol     ratio
1  -1000.0        a1     1.0 -2.0     3.0     1.0     0.0     2.0  0.666667
2  -1000.0        a2     3.0  2.0     4.0     0.0     1.0     1.0  0.250000
Cj     NaN       NaN    -2.0  1.0     3.0 -1000.0 -1000.0     0.0       NaN
Zj     NaN       NaN -4000.0  0.0 -7000.0 -1000.0 -1000.0 -3000.0       NaN
value of objective function is  -3000.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x3
Leaving variable is  a2
simplex table:
       CBi basic_var       x1      x2   x3      a1       a2      sol     ratio
1  -1000.0        a1    -1.25    -3.5  0.0     1.0    -0.75     1.25  0.666667
2      3.0        x3     0.75     0.5  1.0     0.0     0.25     0.25  0.250000
Cj     NaN       NaN    -2.00     1.0  3.0 -1000.0 -1000.00     0.00       NaN
Zj     NaN       NaN  1252.25  3501.5  3.0 -1000.0   750.75 -1249.25       NaN
value of objective 